# Relatorio Técnico Referente a Matéria de Técnicas de Pesquisa Experimental
  
**Tema:** Analise da Variação de Parâmetros na Impressão 3D de Filamentos Poliméricos  
**Profº:** Pedro Bastos Costa  
**Universidade Federal de Minas Gerais - UFMG**

**Membros:**
* Guilherme de Paula Rúbio  
* Matheus Ungaretti Borges  
* Daniel Affonso Vasconcelos  

## *Importação de Bibliotecas*

In [1]:
import numpy as np 
import pandas as pd 
import scipy.stats as st 
import matplotlib.pyplot as plt 

from math import sqrt

%matplotlib inline

## ***Fuções Auxiliares Criadas***  

* **Função**: Teste de hipótese bicaudal, distribuição F

In [2]:
def F_bicaudal_hypoteses (F :float, alpha :float, dn :float, dd :float):
    '''
        Função de teste de hipótese de um teste bicaudal do tipo F
            Entrada:
               * F: Valor calculado de F_0;
               * alpha: nivel de confiança total;
               * dn: graus de liberdade do numerador;
               * dd: graus de liberdade do denominado.

            Saídas:
               * Impressão das Respostas.

            Retorno: Nenhum
    '''
    min = st.f.ppf(alpha/2, dn, dd)         # Limite Superior
    max = st.f.ppf(1-(alpha/2), dn, dd)     # Limite Inferior

    print('O valor da estatísca cálcula F0 é: {:.4f}'.format(F))
    print('O limites da distribuição para {} de {:.2f} é: [{:.4f}:{:.4f}]'.format('\u0251', alpha, min, max))
    print('Portanto,',end=' ')

    if (F<min) or (F>max):
        print('\033[33mREJEITA-SE\033[m a hipótese nula!')
        print('p-valor: {:.4f}'.format( st.f.cdf(F, dn, dd) if (F<min) else (1-st.f.cdf(F, dn, dd)) ))
    else:
        print('\033[31mFALHA EM REJEITAR\033[m a hipótese nula!')

## *Importação dos dados*
Todos os dados utilizados nessa análise estão disponibilizados na plataforma Kaggle. Caso queria acessar a fonte dos dados utilizados basta [clicar aqui.](https://www.kaggle.com/afumetto/3dprinter?select=data.csv)  

### Contexto  
Os dados utilizados são oriundos de uma pesquisa desenvolvida pelo Departamento de Engenharia Mecânica da TR/Selcuk University.  
Essa pesquisa visava avaliar quais os parâmetros de impressão interferem na qualidade de impressão de peças, precisão e rigidez.  
Nos dados apresentados existem nove parâmetro de configuração (entradas), e as medições de três parâmetros de saída.  

### Materiais e Métodos  
* **Impressora:** Ultimaker S5 3-D 
* **Teste de materiais e resistência:** Sincotec GMBH, capacidade de tração - 20kN.

### Conteúdo dos dados
#### Entradas - Parâmetros de configuração:  
* Altura de camada (*Layer Height*) \[mm]  
* Espessura de casca (*Wall Thickness*) \[mm]  
* Densidade de preenchimento (*Infill Density*) \[%]  
* Padrão de preenchimento (*Infill Pattern*)  
* Temperatura do bico de extrusão (*Nozzle Temperature*) \[°C]  
* Temperetura da mesa de impressão (*Bed Temperature*) \[°C]  
* Velocidade de impressão (*Print Speed*) \[mm/s]  
* Material (*Material*)  
* Velocidade do vetilador da extrusora (*Fan Speed*) \[%]  

#### Saídas  
* Rugosidade (*Roughness*) \[µm]
* Tensão de ruptura (*Ultimate Tension Strenght*) \[MPa]
* Alongação (*Elongation*) \[%]  

Abaixo importamos os dados para a análise.  

In [3]:
dados = pd.read_csv('datasets\data_3D_print.csv')   # Importa o DataFrame
dados.head()     # Apresenta as 5 primeiras linhas do DataFrame

,layer_height,wall_thickness,infill_density,infill_pattern,nozzle_temperature,bed_temperature,print_speed,material,fan_speed,roughness,tension_strenght,elongation
0,0.02,8,90,grid,220,60,40,abs,0,25,18,1.2
1,0.02,7,90,honeycomb,225,65,40,abs,25,32,16,1.4
2,0.02,1,80,grid,230,70,40,abs,50,40,8,0.8
3,0.02,4,70,honeycomb,240,75,40,abs,75,68,10,0.5
4,0.02,6,90,grid,250,80,40,abs,100,92,5,0.7


## *Análise das Saídas*  
Como temos três saidas iremos analisa-las separadamente.  

### Saida Rugosidade:  

#### ***Influência do material***  
Umas das primeiras hipóteses que desejamos saber do ponto de vista da rugosidade é se o material afeta estatisticamente seu valor, para isso separaremos as saídas de rugosidade por material para realizar uma análise de experimento com um único fator.

In [4]:
dados_roughness_material = dados[['material','roughness']]
dados_roughness_material.head()

,material,roughness
0,abs,25
1,abs,32
2,abs,40
3,abs,68
4,abs,92


Com os dados separados podemos observar que nosso problema é dividido em dois materiais, ou seja, dois níveis (PLA e ABS). Primeiramente vamos analisar quantas observações de cada materiais nos temos.  

In [5]:
dados_roughness_material['material'].value_counts()

pla    25
abs    25
Name: material, dtype: int64

Como podemos observa o número de observações de cada material é igual.
***
#### Teoria 📖:  
Supomos que nosso valor de saída rugosidade varia de acordo com o material utilizado, o que significaria que ela teria o seguinte comportamento:  
$$
\mathsf{
    Y_{ij}= \mu + \tau_{i} + \epsilon_{ij}
}
\left\{\displaystyle
    \begin{array}{l}
        \mathsf{i=0, 1}\\
        \mathsf{j=0, 1, 2, 3, \dots, 25}
    \end{array}
\right.
$$  
Onde:
   * $\mu$: média global;  
   * $\tau_{i}$: fator de alteração referente ao material utilizado;
   * $\epsilon_{ij}$: erro aleatorio da medição  
***  
Para garantirmos que o realmente o material interfere na rugosidade medida nas nossas peças temos que garantir que o valor de $\tau$ de cada material seja diferente de zero. Logo temos que fazer um teste assumindo as seguintes hipóteses:  
$$
\left\{
    \begin{array}{}
        \mathsf{\mathit{H_0}: \tau_{0}=\tau_{1}=0}\\
        \mathsf{\mathit{h_1}: \tau_{0}\not=\tau_{1}\not=0}
    \end{array}
\right.
$$  
Para isso faremos um teste ANOVA. Para isso assumimos que o modelo assumi uma estatística F, assim:  
$$
\mathsf{
    F_0~=~\dfrac{\dfrac{SQ_{tratamentos}}{a-1}}{\dfrac{SQ_{E}}{a\cdot\left(n-1\right)}}=
\dfrac{\dfrac{n \cdot \displaystyle\sum_{i=1}^{a=2}\left(\bar{y}_{i\cdot} - \bar{y}_{\cdot\cdot}\right)^2}{a-1}}{\dfrac{\displaystyle\sum_{i=1}^{a=2} \sum_{j=1}^{n=25}\left(y_{ij}-\bar{y}_{i\cdot}\right)^2}{a\cdot\left(n-1\right)}}=
    \dfrac{MQ_{tratamentos}}{MQ_E}
}
$$  

Para iniciarmos os calculos vamos primeiramente calular as média e armazena-las

In [6]:
mean_roug_mat = dados_roughness_material.groupby('material').mean()   # media da rugosidade por material
mean_roug_mat

,roughness
material,
abs,193.44
pla,147.72


In [7]:
mean_roug_mat_gen = dados_roughness_material['roughness'].mean()  # Média Geral 
mean_roug_mat_gen

170.58

Calculando o $SQ_{tratamentos}$:

In [8]:
n = (dados_roughness_material['material'].value_counts()).loc['abs']
SQ_trat = n * sum((mean_roug_mat['roughness']-mean_roug_mat_gen)**2)
SQ_trat

26128.980000000003

Calculando $SQ_E$

In [9]:
SQ_abs = sum((dados_roughness_material[dados_roughness_material['material']=='abs']['roughness']-float(mean_roug_mat.loc['abs']))**2)
SQ_pla = sum((dados_roughness_material[dados_roughness_material['material']=='pla']['roughness']-float(mean_roug_mat.loc['pla']))**2)

SQ_e = SQ_abs+SQ_pla
SQ_e

454451.19999999995

Com os erros quadráticos calculados podemos fazer o teste de hipótese:  

In [10]:
a = len(mean_roug_mat)

MQ_trat = SQ_trat/(a-1)
MQe = SQ_e/(a*(n-1))

F0 = MQ_trat/MQe

F_bicaudal_hypoteses(F0, 0.05, a-1, a*(n-1))

O valor da estatísca cálcula F0 é: 2.7598
O limites da distribuição para ɑ de 0.05 é: [0.0010:5.3541]
Portanto, FALHA EM REJEITAR a hipótese nula!


Como podemos observar para um $\alpha$ de 0,05 FALHAMOS EM REJEITAR a hipótese nula, logo o material não afeta estatísticamente no valor de rugosidade.   
Além do teste ANOVA utilizaremos então o método MDS para avaliar se a as médias medidas tem diferença significativas na rugosidade.  
Para isso temos que para as médias terem diferenças significativas devemos relacionar:  
$$
\mathsf{
    |\bar{y}_{i\cdot}-\bar{y}_{j\cdot}|> MDS
}
$$  
$$
\mathsf{
    |\bar{y}_{i\cdot}-\bar{y}_{j\cdot}|> \mathit{t}_{\frac{\alpha}{2};a \cdot(n-1)} \cdot \sqrt{\dfrac{2\cdot MQ_E}{n}}
}
$$  
Assim:  

In [11]:
MDS = st.t.ppf(1-0.025,a*(n-1)) * sqrt((2*MQe)/n)

print('{:^10}|{:^10}'.format('abs-pla','MDS'))
print('{:^1}{:.4f}{:^2}|{:^1}{:.4f}{:^2}'.format('',float(mean_roug_mat.loc['abs'] - mean_roug_mat.loc['pla']),'','', MDS,''))

 abs-pla  |   MDS    
 45.7200  | 55.3352  


Analisando a mínima diferença significativa (MDS), vemos que realmente a mudança do material não afeta estatistiamente o valor da rugosidade.  
Essa conclusão era esperada, uma vez que como a impressão ocorre por camadas, a rugosidade deve ser afeta prinicpalmente por parâmetros que determinam as caracteristicas dessas camadas que o material em si.  